In [ ]:
from plot_iver import prepare_data, plot_inputs_outputs, plot_elevator_rudder_v_roll, parse_config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List
from pathlib import Path

In [ ]:
keys = [ # Config parameters we want to extract from the config file
    "control.depthpidd",
    "control.depthpidi",
    "control.depthpidp",
    "control.headingdeadband",
    "control.headingpidd",
    "control.headingpidi",
    "control.headingpidp",
    "control.rollpidp",
    "control.rollpidi",
    "control.rollpidd",
    "control.rolldeadband"
]

def process_run(log_file, config_file):
    """Function to produce descriptive plot for a single recorded vehicle run."""

    try:
        df = prepare_data(log_file) # extract the data
    except (pd.errors.EmptyDataError, TypeError) as e:
        return
        
    config = parse_config(config_file, keys) # parse the PID parameters from the file
    
    # Save plot of inputs and outputs in a folder
    fig, axs = plot_inputs_outputs(df)
    fig.suptitle(f'Depth PID: {config["control.depthpidp"], config["control.depthpidi"], config["control.depthpidd"]}. Heading PID: {config["control.headingpidp"], config["control.headingpidi"], config["control.headingpidd"]} Roll PID: {config["control.rollpidp"], config["control.rollpidi"], config["control.rollpidd"]} Roll Deadband: {config["control.rolldeadband"]}\n{log_file}')
    fig.tight_layout()

    plt.savefig("plots/input_output/" + str(Path(log_file).name) + ".png", dpi=400)

    # Save plot of elevator, rudder, and roll rate in a different folder
    fig, axs = plot_elevator_rudder_v_roll(df)
    fig.suptitle(f'Depth PID: {config["control.depthpidp"], config["control.depthpidi"], config["control.depthpidd"]}. Heading PID: {config["control.headingpidp"], config["control.headingpidi"], config["control.headingpidd"]} Roll PID: {config["control.rollpidp"], config["control.rollpidi"], config["control.rollpidd"]} Roll Deadband: {config["control.rolldeadband"]}\n{log_file}')
    fig.tight_layout()
    
    plt.savefig("plots/rollrate/" + str(Path(log_file).name) + ".png", dpi=400)

In [ ]:
# Look through all of the log files and attempt to plot them
root_directory = Path(".")
for path_object in root_directory.glob('**/*IverControlData*.log'):
    print(f"Processing path: {path_object}")

    # Extract timestamp
    s = path_object.name
    timestamp = s[s.find("2022"):s.find(".log")]

    # Find corresponding config
    config_file = next(root_directory.glob(f'**/*Config-{timestamp}.log*')).absolute()

    # Apply the processing function from above
    process_run(log_file=path_object, config_file=config_file)